## Imports

In [1]:
import pandas as pd
import os
from openpyxl import Workbook
from simplified_scrapy import SimplifiedDoc, utils

from collections import defaultdict
pd.set_option('display.float_format', '{:.2f}'.format)

## Morningstar dataset conversion to Excel file (.xlsx)

In [2]:
"""
import os
files = []
for i in os.walk('./datasets'):
    files= i[2]
    
print(files)
"""

"\nimport os\nfiles = []\nfor i in os.walk('./datasets'):\n    files= i[2]\n    \nprint(files)\n"

In [3]:
"""
def readFile(filename):
    xml = utils.getFileContent(filename)
    doc = SimplifiedDoc(xml)
    tables = doc.selects('Worksheet').selects('Row').selects('Cell').text # Get all data
    sheetNames = doc.selects('Worksheet>ss:Name()') # Get sheet name
    return sheetNames,tables,filename.split('/')[2].split('.')[0]

def to_Excel(sheetNames,tables, filename):
    wb = Workbook() # Create Workbook
    for i in range(len(sheetNames)):
        worksheet = wb.create_sheet(sheetNames[i]) # Create sheet
        for row in tables[i]:
            worksheet.append(row)
    print('./datasets_excel/'+ filename + '.xlsx')
    wb.save('./datasets_excel/'+ filename + '.xlsx') # Save file
"""

"\ndef readFile(filename):\n    xml = utils.getFileContent(filename)\n    doc = SimplifiedDoc(xml)\n    tables = doc.selects('Worksheet').selects('Row').selects('Cell').text # Get all data\n    sheetNames = doc.selects('Worksheet>ss:Name()') # Get sheet name\n    return sheetNames,tables,filename.split('/')[2].split('.')[0]\n\ndef to_Excel(sheetNames,tables, filename):\n    wb = Workbook() # Create Workbook\n    for i in range(len(sheetNames)):\n        worksheet = wb.create_sheet(sheetNames[i]) # Create sheet\n        for row in tables[i]:\n            worksheet.append(row)\n    print('./datasets_excel/'+ filename + '.xlsx')\n    wb.save('./datasets_excel/'+ filename + '.xlsx') # Save file\n"

In [4]:
"""
for filename in (os.listdir("./datasets")):
        name = filename.split('.')[0]
        print(name)
        
        dst = name + ".xml"
        src ='./datasets/'+ filename
        dst ='./datasets/'+ dst
          
        # rename() function will
        # rename all the files
        os.rename(src, dst)
"""

'\nfor filename in (os.listdir("./datasets")):\n        name = filename.split(\'.\')[0]\n        print(name)\n        \n        dst = name + ".xml"\n        src =\'./datasets/\'+ filename\n        dst =\'./datasets/\'+ dst\n          \n        # rename() function will\n        # rename all the files\n        os.rename(src, dst)\n'

In [5]:
"""
for file in files:
    to_Excel(*readFile("./datasets/" + file))
"""

'\nfor file in files:\n    to_Excel(*readFile("./datasets/" + file))\n'

## Import datasets into Pandas dataframes

In [6]:
files = []
for i in os.walk('./datasets_excel/'):
    files=i[2]

workbooks = {}
for file in files:
    workbooks[file.split('.')[0]] = pd.read_excel('./datasets_excel/' + file, sheet_name=None)

In [7]:
print(list(workbooks.keys()))

['ADH_FY', 'ADH_HY', 'ANZ_FY', 'ANZ_HY', 'BCI_FY', 'BCI_HY', 'BFG_FY', 'BFG_HY', 'BHP_FY', 'BHP_HY', 'BNL_FY', 'BNL_HY', 'BPG_FY', 'BPG_HY', 'BTR_FY', 'BTR_HY', 'CBA_FY', 'CBA_HY', 'CCC_FY', 'CCC_HY', 'CIM_FY', 'CIM_HY', 'CVR_FY', 'CVR_HY', 'DGX_FY', 'DGX_HY', 'DTZ_FY', 'DTZ_HY', 'GMN_FY', 'GMN_HY', 'HWK_FY', 'HWK_HY', 'JBH_FY', 'JBH_HY', 'LCT_FY', 'LCT_HY', 'MEB_FY', 'MEB_HY', 'MFG_FY', 'MFG_HY', 'MGC_FY', 'MGC_HY', 'MQG_FY', 'MQG_HY', 'NAB_FY', 'NAB_HY', 'NCM_FY', 'NCM_HY', 'NGI_FY', 'NGI_HY', 'REX_FY', 'REX_HY', 'RIO_FY', 'RIO_HY', 'RNO_FY', 'RNO_HY', 'SRX_FY', 'SRX_HY', 'SRZ_FY', 'SRZ_HY', 'WBC_FY', 'WBC_HY', 'WCC_FY', 'WCC_HY']


## Cleaning

### Normalising statement items

In [8]:
statement_items = defaultdict(set)

for workbook in workbooks:
    for sheet in workbooks[workbook]:
        if sheet != 'Sheet' and 'Item' in workbooks[workbook][sheet]:
            statement_items[sheet].update(workbooks[workbook][sheet]['Item'])
            
statement_items.keys()

dict_keys(['Profit Loss', 'Balance Sheet', 'Cash Flow', 'Per Share Statisticts', 'Sundry Analysis', 'Growth Rates', 'Ratio Analysis', 'Asset Base Analysis'])

### Combining HY and FY datasets

In [9]:
workbooks_combined = defaultdict(dict)
workbooks_keys = list(workbooks.keys())

for i in range(len(workbooks)//2):
    for sheet in statement_items:
        new_sheet = _
        if sheet in workbooks[workbooks_keys[i*2]]:
            new_sheet = pd.concat([workbooks[workbooks_keys[i*2]][sheet], workbooks[workbooks_keys[i*2 + 1]][sheet]], axis=1, join='inner')
            new_sheet = new_sheet.loc[:,~new_sheet.columns.duplicated()]
            new_sheet = new_sheet.sort_index(axis=1)
            idx = pd.to_datetime(new_sheet.columns, errors='coerce', format='%m/%y').argsort()
            new_sheet = new_sheet.iloc[:, list(idx[-3:]) + list(idx[:-3])]
            new_sheet = new_sheet.set_index(['ASX Code', 'Company Name', 'Item'])
            workbooks_combined[workbooks_keys[i*2].split('_')[0]][sheet] = new_sheet    
        else:
            print("Missing tab:", workbooks_keys[i*2], sheet)            

Missing tab: ANZ_FY Sundry Analysis
Missing tab: CBA_FY Sundry Analysis
Missing tab: NAB_FY Sundry Analysis
Missing tab: WBC_FY Sundry Analysis


In [10]:
workbooks_combined.keys()

dict_keys(['ADH', 'ANZ', 'BCI', 'BFG', 'BHP', 'BNL', 'BPG', 'BTR', 'CBA', 'CCC', 'CIM', 'CVR', 'DGX', 'DTZ', 'GMN', 'HWK', 'JBH', 'LCT', 'MEB', 'MFG', 'MGC', 'MQG', 'NAB', 'NCM', 'NGI', 'REX', 'RIO', 'RNO', 'SRX', 'SRZ', 'WBC', 'WCC'])

In [11]:
workbooks_combined['ADH']['Profit Loss']

12/14  \
ASX Code Company Name   Item                                               
ADH      Adairs Limited Operating Revenue                             --   
                        Other Revenue                                 --   
                        Total Revenue Excluding Interest              --   
                        Operating Expenses                            --   
                        EBITDA                                         0   
                        Depreciation                                  --   
                        Amortisation                                  --   
                        Depreciation and Amortisation                 --   
                        EBIT                                           0   
                        Interest Revenue                              --   
                        Interest Expense                              --   
                        Net Interest Expense                          --   
                        PreTax Profit                                 --   
                        Tax Expense                                    0   
                        Net Profit after Tax Before Abnormals          0   
                        Abnormals                                      0   
                        Abnormals Tax                                  0   
                        Net Abnormals                                  0   
                        Reported NPAT After Abnormals                  0   
                        Outside Equity Interests                       0   
                        Shares Outstanding at Period End       165874785   
                        Weighted Average Number of Shares              0   
                        EPS Adjusted (cents/share)                  0.00   
                        EPS After Abnormals (cents/share)           0.00   

                                                                      06/15  \
ASX Code Company Name   Item                                                  
ADH      Adairs Limited Operating Revenue                      210878000.00   
                        Other Revenue                             166000.00   
                        Total Revenue Excluding Interest       211044000.00   
                        Operating Expenses                    -188019000.00   
                        EBITDA                                  23025000.00   
                        Depreciation                            -3858000.00   
                        Amortisation                             -728000.00   
                        Depreciation and Amortisation           -4586000.00   
                        EBIT                                    18439000.00   
                        Interest Revenue                          445000.00   
                        Interest Expense                       -14267000.00   
                        Net Interest Expense                   -13822000.00   
                        PreTax Profit                            4617000.00   
                        Tax Expense                             -1675000.00   
                        Net Profit after Tax Before Abnormals    2942000.00   
                        Abnormals                                      0.00   
                        Abnormals Tax                                  0.00   
                        Net Abnormals                                  0.00   
                        Reported NPAT After Abnormals             745000.00   
                        Outside Equity Interests                       0.00   
                        Shares Outstanding at Period End       165875000.00   
                        Weighted Average Number of Shares      151471000.00   
                        EPS Adjusted (cents/share)                     1.94   
                        EPS After Abnormals (cents/share)              0.49   

                                   